# Normalization

Import libraries and functions.

In [1]:
import pandas as pd
import numpy as np
import glob
import os
from zipfile import ZipFile
import warnings
warnings.filterwarnings("ignore")
import functools as ft
import ipywidgets as widgets
from ipywidgets import Layout
from ipywidgets import interact, interact_manual
from pandas.api.types import is_numeric_dtype

Taking as reference both works of https://www.pluralsight.com/guides/cleaning-up-data-from-outliers and https://careerfoundry.com/en/blog/data-analytics/how-to-find-outliers/, for normalizing our data we need to start computing the outliers and removing them from our dataframe. As there is not a direct function of pandas that performs this step, it´s been step-by-step code, where we begin with the computation of the quartiles, then the IQR (Inter Quartile Range) and finally the upper and lower limit.

##### IQR explanation

The interquartile range (IQR) measures the spread of the middle half of your data. It is the range for the middle 50% of your sample. Use the IQR to assess the variability where most of your values lie. Larger values indicate that the central portion of your data spread out further. Conversely, smaller values show that the middle values cluster more tightly.

To visualize the interquartile range, imagine dividing your data into quarters. Statisticians refer to these quarters as quartiles and label them from low to high as Q1, Q2, Q3, and Q4. The lowest quartile (Q1) covers the smallest quarter of values in your dataset. The upper quartile (Q4) comprises the highest quarter of values. The interquartile range is the middle half of the data that lies between the upper and lower quartiles. In other words, the interquartile range includes the 50% of data points that are above Q1 and below Q4. The IQR is the red area in the graph below, containing Q2 and Q3 (not labeled).

https://camo.githubusercontent.com/a5f6cf8164048f8c28f9b00b94e1264480c8c3b20a1b3d0bdca47083f3a86a19/68747470733a2f2f69302e77702e636f6d2f7374617469737469637362796a696d2e636f6d2f77702d636f6e74656e742f75706c6f6164732f323031382f30332f696e7465727175617274696c655f72616e67652e706e673f773d3537362673736c3d31

When measuring variability, statisticians prefer using the interquartile range instead of the full data range because extreme values and outliers affect it less. Typically, use the IQR with a measure of central tendency, such as the median, to understand your data’s center and spread. This combination creates a fuller picture of your data’s distribution.

Therefore it is being utilized to get rid of all the outliers that may come from errors when creating the data or from unexpected years.

Firstly, we compute the first quartile (Q1=25%) and the third quartile (Q3=75%). For that, we have grouped the data by country code and indicator name, so we get the Q1 and Q3 values for each indicator in each geographical area. 

In [11]:
grouped=BronzeDataFrame.groupby(['Country Code','Indicator Name'])
grouped

In [12]:
Q1=BronzeDataFrame.groupby(['Country Code','Indicator Name']).quantile(0.25)
Q3=BronzeDataFrame.groupby(['Country Code','Indicator Name']).quantile(0.75)
IQR=Q3-Q1
IQR

Date  \
Country Code Indicator Name                                             
ARE          Access to clean fuels and technologies for cook...  10.0   
             Access to clean fuels and technologies for cook...  10.0   
             Access to clean fuels and technologies for cook...  10.0   
             Access to electricity (% of population)             15.0   
             Access to electricity, rural (% of rural popula...  15.0   
...                                                               ...   
ZAF          Women who believe a husband is justified in bea...   0.0   
             Women who were first married by age 15 (% of wo...   9.0   
             Women who were first married by age 18 (% of wo...   9.0   
             Women's share of population ages 15+ living wit...  15.0   
             Young people (ages 15-24) newly infected with HIV   15.0   

                                                                    Value  
Country Code Indicator Name                                                
ARE          Access to clean fuels and technologies for cook...      0.00  
             Access to clean fuels and technologies for cook...      0.00  
             Access to clean fuels and technologies for cook...      0.00  
             Access to electricity (% of population)                 0.00  
             Access to electricity, rural (% of rural popula...      0.00  
...                                                                   ...  
ZAF          Women who believe a husband is justified in bea...      0.00  
             Women who were first married by age 15 (% of wo...      0.15  
             Women who were first married by age 18 (% of wo...      2.15  
             Women's share of population ages 15+ living wit...      4.90  
             Young people (ages 15-24) newly infected with HIV   85000.00  

[59239 rows x 2 columns]

Once we got the quartiles, we compute the upper and lower limit, with a basic mathematical expression.

In [13]:
lower_limit=Q1 - 1.5 * IQR
lower=lower_limit.drop(['Date'],axis=1)
lower.rename(columns={"Value":"Lower limit"})

Lower limit
Country Code Indicator Name                                                 
ARE          Access to clean fuels and technologies for cook...      100.000
             Access to clean fuels and technologies for cook...      100.000
             Access to clean fuels and technologies for cook...      100.000
             Access to electricity (% of population)                 100.000
             Access to electricity, rural (% of rural popula...      100.000
...                                                                      ...
ZAF          Women who believe a husband is justified in bea...        1.000
             Women who were first married by age 15 (% of wo...        0.625
             Women who were first married by age 18 (% of wo...        1.375
             Women's share of population ages 15+ living wit...       49.850
             Young people (ages 15-24) newly infected with HIV    -22500.000

[59239 rows x 1 columns]

In [14]:
upper_limit=Q3 + 1.5 * IQR
upper=upper_limit.drop(['Date'],axis=1)
upper.rename(columns={"Value":"Upper limit"})

Upper limit
Country Code Indicator Name                                                 
ARE          Access to clean fuels and technologies for cook...      100.000
             Access to clean fuels and technologies for cook...      100.000
             Access to clean fuels and technologies for cook...      100.000
             Access to electricity (% of population)                 100.000
             Access to electricity, rural (% of rural popula...      100.000
...                                                                      ...
ZAF          Women who believe a husband is justified in bea...        1.000
             Women who were first married by age 15 (% of wo...        1.225
             Women who were first married by age 18 (% of wo...        9.975
             Women's share of population ages 15+ living wit...       69.450
             Young people (ages 15-24) newly infected with HIV    317500.000

[59239 rows x 1 columns]

Thirdly, we join the three tables we have (main dataframe, upper limit and lower limit) by matching country code and indicator name..

In [15]:
dfs = [BronzeDataFrame,lower,upper]
df_joined = ft.reduce(lambda left, right: pd.merge(left, right, on=['Country Code','Indicator Name']), dfs)
df_joined

,Country Code,Indicator Name,Date,Value_x,Value_y,Value
0,DZA,Access to clean fuels and technologies for coo...,2000,97.1,97.0,101.0
1,DZA,Access to clean fuels and technologies for coo...,2001,97.3,97.0,101.0
2,DZA,Access to clean fuels and technologies for coo...,2002,97.8,97.0,101.0
3,DZA,Access to clean fuels and technologies for coo...,2003,98.0,97.0,101.0
4,DZA,Access to clean fuels and technologies for coo...,2004,98.2,97.0,101.0
...,...,...,...,...,...,...
1225413,YEM,Young people (ages 15-24) newly infected with HIV,2016,200.0,-50.0,350.0
1225414,YEM,Young people (ages 15-24) newly infected with HIV,2017,200.0,-50.0,350.0
1225415,YEM,Young people (ages 15-24) newly infected with HIV,2018,200.0,-50.0,350.0
1225416,YEM,Young people (ages 15-24) newly infected with HIV,2019,200.0,-50.0,350.0


In [16]:
list(df_joined)

['Country Code', 'Indicator Name', 'Date', 'Value_x', 'Value_y', 'Value']

We rename the columns of the new table, as the columns headers are not saved after the joining. 

In [17]:
renamed=df_joined.set_axis(['Country','Indicator','Year', 'Real value', 'Lower value', 'Upper value'], axis=1, inplace=False)
renamed

,Country,Indicator,Year,Real value,Lower value,Upper value
0,DZA,Access to clean fuels and technologies for coo...,2000,97.1,97.0,101.0
1,DZA,Access to clean fuels and technologies for coo...,2001,97.3,97.0,101.0
2,DZA,Access to clean fuels and technologies for coo...,2002,97.8,97.0,101.0
3,DZA,Access to clean fuels and technologies for coo...,2003,98.0,97.0,101.0
4,DZA,Access to clean fuels and technologies for coo...,2004,98.2,97.0,101.0
...,...,...,...,...,...,...
1225413,YEM,Young people (ages 15-24) newly infected with HIV,2016,200.0,-50.0,350.0
1225414,YEM,Young people (ages 15-24) newly infected with HIV,2017,200.0,-50.0,350.0
1225415,YEM,Young people (ages 15-24) newly infected with HIV,2018,200.0,-50.0,350.0
1225416,YEM,Young people (ages 15-24) newly infected with HIV,2019,200.0,-50.0,350.0


Now that we have the table correctly defined, we remove from our dataframe the values that are outside our range, as it means that they are outliers.

In [18]:
sin_outliers=renamed.loc[~((renamed['Real value']<renamed['Lower value']) | (renamed['Real value']>renamed['Upper value']))]
sin_outliers

,Country,Indicator,Year,Real value,Lower value,Upper value
0,DZA,Access to clean fuels and technologies for coo...,2000,97.1,97.0,101.0
1,DZA,Access to clean fuels and technologies for coo...,2001,97.3,97.0,101.0
2,DZA,Access to clean fuels and technologies for coo...,2002,97.8,97.0,101.0
3,DZA,Access to clean fuels and technologies for coo...,2003,98.0,97.0,101.0
4,DZA,Access to clean fuels and technologies for coo...,2004,98.2,97.0,101.0
...,...,...,...,...,...,...
1225413,YEM,Young people (ages 15-24) newly infected with HIV,2016,200.0,-50.0,350.0
1225414,YEM,Young people (ages 15-24) newly infected with HIV,2017,200.0,-50.0,350.0
1225415,YEM,Young people (ages 15-24) newly infected with HIV,2018,200.0,-50.0,350.0
1225416,YEM,Young people (ages 15-24) newly infected with HIV,2019,200.0,-50.0,350.0


From the data above, we can perceive that our data comes down from 1225418  rows to 1189068, so 36.350  were outliers. The next steps are to order and display data better, removing those columns that we just do not need and pivoting the rows and columns. 

In [19]:
df_limpio=sin_outliers.drop(['Lower value','Upper value'],axis=1)
df_limpio

,Country,Indicator,Year,Real value
0,DZA,Access to clean fuels and technologies for coo...,2000,97.1
1,DZA,Access to clean fuels and technologies for coo...,2001,97.3
2,DZA,Access to clean fuels and technologies for coo...,2002,97.8
3,DZA,Access to clean fuels and technologies for coo...,2003,98.0
4,DZA,Access to clean fuels and technologies for coo...,2004,98.2
...,...,...,...,...
1225413,YEM,Young people (ages 15-24) newly infected with HIV,2016,200.0
1225414,YEM,Young people (ages 15-24) newly infected with HIV,2017,200.0
1225415,YEM,Young people (ages 15-24) newly infected with HIV,2018,200.0
1225416,YEM,Young people (ages 15-24) newly infected with HIV,2019,200.0


In [20]:
cols=df_limpio['Indicator'].unique().tolist()

In [21]:
SilverDataFrame=df_limpio.set_index(["Country", "Year"]).pivot(columns="Indicator", values="Real value").reset_index()
SilverDataFrame

Indicator,Country,Year,ARI treatment (% of children under 5 taken to a health provider),Access to clean fuels and technologies for cooking (% of population),"Access to clean fuels and technologies for cooking, rural (% of rural population)","Access to clean fuels and technologies for cooking, urban (% of urban population)",Access to electricity (% of population),"Access to electricity, rural (% of rural population)","Access to electricity, urban (% of urban population)",Account ownership at a financial institution or with a mobile-money-service provider (% of population ages 15+),...,Women who believe a husband is justified in beating his wife (any of five reasons) (%),Women who believe a husband is justified in beating his wife when she argues with him (%),Women who believe a husband is justified in beating his wife when she burns the food (%),Women who believe a husband is justified in beating his wife when she goes out without telling him (%),Women who believe a husband is justified in beating his wife when she neglects the children (%),Women who believe a husband is justified in beating his wife when she refuses sex with him (%),Women who were first married by age 15 (% of women ages 20-24),Women who were first married by age 18 (% of women ages 20-24),Women's share of population ages 15+ living with HIV (%),Young people (ages 15-24) newly infected with HIV
0,ARE,1990,NaN,NaN,NaN,NaN,100.000000,100.000000,100.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.8,100.0
1,ARE,1991,NaN,NaN,NaN,NaN,100.000000,100.000000,100.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.2,100.0
2,ARE,1992,NaN,NaN,NaN,NaN,100.000000,100.000000,100.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.4,100.0
3,ARE,1993,NaN,NaN,NaN,NaN,100.000000,100.000000,100.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,100.0
4,ARE,1994,NaN,NaN,NaN,NaN,100.000000,100.000000,100.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1531,ZAF,2017,NaN,85.2,64.6,94.20,84.400002,76.738983,88.373024,69.218491,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63.3,100000.0
1532,ZAF,2018,NaN,85.7,65.5,94.65,84.699997,77.168495,88.518814,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63.7,92000.0
1533,ZAF,2019,NaN,86.3,65.5,94.90,85.000000,77.611824,88.662704,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64.1,85000.0
1534,ZAF,2020,NaN,86.8,65.9,95.20,84.385536,75.264854,88.806267,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64.4,79000.0


On the other hand, another big stone of normalizations is to nan/null values, which we have in all variables.

In [22]:
SilverDataFrame.isna().sum().sum()

1016628

As we can observe, we have lots of missing data, and as there is no optimal way to fullfill these values, thus, we will test some to arrive to the optimal method for our data set.

First, we need to create some lists so our loops work.

In [23]:
df=SilverDataFrame
europe_list=['DEU','FRA','SWE','GBR','ESP','HRV','POL','GRC','AUT','NLD']
persian_list=['IRQ','QAT','ARE','SAU','AZE','YEM','OMN']
naf_list=['DZA','EGY','LBY','ISR','TUR','MAR']
saf_list=['SEN','ZAF','LBR','MOZ','CMR','NGA','GHA']
asia_list=['BGD','IND','VNM','THA','IDN','PHL','KOR']
latam_list=['MEX','BRA','ARG','PER','VEN','COL','CHL','PAN','CRI']
two_list=['USA','CHN']
country_list=europe_list+persian_list+naf_list+saf_list+asia_list+latam_list+two_list


We are attempting the linear interpolation, which is achieved by geometrically rendering a straight line between two adjacent points on a graph or plane.

In [24]:
dat=df.loc[df.loc[:, 'Country'] == country_list[0]]
datc=dat.interpolate(method="linear")
data=datc

for i in range(1,len(country_list)):
    dat=df.loc[df.loc[:, 'Country'] == country_list[i]]
    datc=dat.interpolate(method="linear")
    data=pd.concat((data, datc), axis = 0)
data.isna().sum().sum()

685787

Here we attempt the backward filling, filling the previous cell with future values.

In [25]:
dat=df.loc[df.loc[:, 'Country'] == country_list[0]]
datc=dat.fillna(method='bfill')
data=datc

for i in range(1,len(country_list)):
    dat=df.loc[df.loc[:, 'Country'] == country_list[i]]
    datc=dat.fillna(method='bfill')
    data=pd.concat((data, datc), axis = 0)
data.isna().sum().sum()

498648

Here we will attempt the forward filling, which concists of filling the next cell with previous values.

In [26]:
dat=df.loc[df.loc[:, 'Country'] == country_list[0]]
datc=dat.fillna(method='ffill')
data=datc

for i in range(1,len(country_list)):
    dat=df.loc[df.loc[:, 'Country'] == country_list[i]]
    datc=dat.fillna(method='ffill')
    data=pd.concat((data, datc), axis = 0)
data.isna().sum().sum()

685787

The linear interpolation a form of interpolation, which involves the generation of new values based on an existing set of values. Linear interpolation is achieved by geometrically rendering a straight line between two adjacent points on a graph or plane. Whereas the backwards filling, will help us to arrive to those values which have not been fullfilled with the linear interpolation.

And as none of the methods have worked out correctly, independently, we are going to mix them, to achieve a better result.

In [27]:
dat=df.loc[df.loc[:, 'Country'] == country_list[0]]
datc=dat.interpolate(method="linear")
datc=datc.fillna(method='ffill')
data=datc

for i in range(1,len(country_list)):
    dat=df.loc[df.loc[:, 'Country'] == country_list[i]]
    datc=dat.interpolate(method="linear")
    datc=datc.fillna(method='ffill')
    data=pd.concat((data, datc), axis = 0)
data.isna().sum().sum()

685787

In [28]:
dat=df.loc[df.loc[:, 'Country'] == country_list[0]]
datc=dat.interpolate(method="linear")
datc=datc.fillna(method='bfill')
data=datc

for i in range(1,len(country_list)):
    dat=df.loc[df.loc[:, 'Country'] == country_list[i]]
    datc=dat.interpolate(method="linear")
    datc=datc.fillna(method='bfill')
    data=pd.concat((data, datc), axis = 0)
data.isna().sum().sum()

310048

And finally, mixing the three methods all together.

In [29]:
dat=df.loc[df.loc[:, 'Country'] == country_list[0]]
datc=dat.interpolate(method="linear")
datf=datc.fillna(method='bfill')
datr=datf.fillna(method='ffill')
data=datr

for i in range(1,len(country_list)):
    dat=df.loc[df.loc[:, 'Country'] == country_list[i]]
    datc=dat.interpolate(method="linear")
    datc=datc.fillna(method='bfill')
    datc=datc.fillna(method='ffill')
    data=pd.concat((data, datc), axis = 0)
data.isna().sum().sum()

310048

##### Conclusion

Therefore, the preferred method for the Nan values´ treatment that we are going to develop is a mix, between the linear interpolation and backwards filling.

In [30]:
dat=df.loc[df.loc[:, 'Country'] == country_list[0]]
datc=dat.interpolate(method="linear")
datf=datc.fillna(method='bfill')
datr=datf.fillna(method='ffill')
data=datr

for i in range(1,len(country_list)):
    dat=df.loc[df.loc[:, 'Country'] == country_list[i]]
    datc=dat.interpolate(method="linear")
    datc=datc.fillna(method='bfill')
    datc=datc.fillna(method='ffill')
    data=pd.concat((data, datc), axis = 0)
data

Indicator,Country,Year,ARI treatment (% of children under 5 taken to a health provider),Access to clean fuels and technologies for cooking (% of population),"Access to clean fuels and technologies for cooking, rural (% of rural population)","Access to clean fuels and technologies for cooking, urban (% of urban population)",Access to electricity (% of population),"Access to electricity, rural (% of rural population)","Access to electricity, urban (% of urban population)",Account ownership at a financial institution or with a mobile-money-service provider (% of population ages 15+),...,Women who believe a husband is justified in beating his wife (any of five reasons) (%),Women who believe a husband is justified in beating his wife when she argues with him (%),Women who believe a husband is justified in beating his wife when she burns the food (%),Women who believe a husband is justified in beating his wife when she goes out without telling him (%),Women who believe a husband is justified in beating his wife when she neglects the children (%),Women who believe a husband is justified in beating his wife when she refuses sex with him (%),Women who were first married by age 15 (% of women ages 20-24),Women who were first married by age 18 (% of women ages 20-24),Women's share of population ages 15+ living with HIV (%),Young people (ages 15-24) newly infected with HIV
352,DEU,1990,NaN,100.0,100.0,100.0,100.0,100.0,100.0,98.133621,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.1,500.0
353,DEU,1991,NaN,100.0,100.0,100.0,100.0,100.0,100.0,98.133621,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.1,500.0
354,DEU,1992,NaN,100.0,100.0,100.0,100.0,100.0,100.0,98.133621,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.1,500.0
355,DEU,1993,NaN,100.0,100.0,100.0,100.0,100.0,100.0,98.133621,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.1,500.0
356,DEU,1994,NaN,100.0,100.0,100.0,100.0,100.0,100.0,98.133621,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.1,500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,CHN,2017,NaN,73.2,55.2,86.2,100.0,100.0,100.0,80.229118,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
252,CHN,2018,NaN,75.6,59.0,87.4,100.0,100.0,100.0,80.229118,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
253,CHN,2019,NaN,77.6,61.9,88.4,100.0,100.0,100.0,80.229118,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
254,CHN,2020,NaN,79.4,65.2,89.4,100.0,100.0,100.0,80.229118,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Now, we will drop the columns which have over X% missing values because the absence of data creates an unreliable source. This % can be adjusted in the following slider. We have predetermined that 20% is a great starting point.

In [31]:
Slider1=widgets.FloatSlider(
    value=0.2,
    min=0,
    max=1.0,
    step=0.05,
    description='% that creates unreliable source:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)
Slider1

FloatSlider(value=0.2, continuous_update=False, description='% that creates unreliable source:', max=1.0, read…

In [32]:
number_1=len(data.index)*Slider1.value
for i in range(0, len(cols)):
    if data[cols[i]].isna().sum()>number_1:
        del(data[cols[i]])
        print(cols[i])
data

Adults (ages 15+) and children (ages 0-14) newly infected with HIV
Adults (ages 15-49) newly infected with HIV
Antiretroviral therapy coverage (% of people living with HIV)
Antiretroviral therapy coverage for PMTCT (% of pregnant women living with HIV)
ARI treatment (% of children under 5 taken to a health provider)
Average transaction cost of sending remittances to a specific country (%)
Average working hours of children, study and work, ages 7-14 (hours per week)
Average working hours of children, study and work, female, ages 7-14 (hours per week)
Average working hours of children, study and work, male, ages 7-14 (hours per week)
Average working hours of children, working only, ages 7-14 (hours per week)
Average working hours of children, working only, female, ages 7-14 (hours per week)
Average working hours of children, working only, male, ages 7-14 (hours per week)
Bank capital to assets ratio (%)
Bank liquid reserves to bank assets ratio (%)
Bank nonperforming loans to total gross

Indicator,Country,Year,Access to clean fuels and technologies for cooking (% of population),"Access to clean fuels and technologies for cooking, rural (% of rural population)","Access to clean fuels and technologies for cooking, urban (% of urban population)",Access to electricity (% of population),"Access to electricity, rural (% of rural population)","Access to electricity, urban (% of urban population)",Account ownership at a financial institution or with a mobile-money-service provider (% of population ages 15+),"Account ownership at a financial institution or with a mobile-money-service provider, female (% of population ages 15+)",...,Urban population growth (annual %),Urban population living in areas where elevation is below 5 meters (% of total population),"Vulnerable employment, female (% of female employment) (modeled ILO estimate)","Vulnerable employment, male (% of male employment) (modeled ILO estimate)","Vulnerable employment, total (% of total employment) (modeled ILO estimate)","Wage and salaried workers, female (% of female employment) (modeled ILO estimate)","Wage and salaried workers, male (% of male employment) (modeled ILO estimate)","Wage and salaried workers, total (% of total employment) (modeled ILO estimate)","Water productivity, total (constant 2015 US$ GDP per cubic meter of total freshwater withdrawal)",Women Business and the Law Index Score (scale 1-100)
352,DEU,1990,100.0,100.0,100.0,100.0,100.0,100.0,98.133621,98.704536,...,1.056365,3.031776,5.700000,4.800000,5.170000,92.050003,89.110001,90.330002,54.519497,71.250
353,DEU,1991,100.0,100.0,100.0,100.0,100.0,100.0,98.133621,98.704536,...,0.934908,3.029378,5.700000,4.800000,5.170000,92.050003,89.110001,90.330002,54.519497,71.250
354,DEU,1992,100.0,100.0,100.0,100.0,100.0,100.0,98.133621,98.704536,...,0.884470,3.026980,5.740000,4.930000,5.270000,91.910004,88.589996,89.970001,54.519497,71.250
355,DEU,1993,100.0,100.0,100.0,100.0,100.0,100.0,98.133621,98.704536,...,0.843967,3.024581,5.850000,5.040000,5.380000,91.669998,88.250000,89.669998,56.039631,71.250
356,DEU,1994,100.0,100.0,100.0,100.0,100.0,100.0,98.133621,98.704536,...,0.636245,3.022183,5.610000,5.200000,5.370000,91.629997,87.739998,89.370003,57.559764,71.250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,CHN,2017,73.2,55.2,86.2,100.0,100.0,100.0,80.229118,76.364731,...,2.739664,4.203002,46.530001,42.949999,44.530002,52.430000,54.169998,53.400002,21.358958,75.625
252,CHN,2018,75.6,59.0,87.4,100.0,100.0,100.0,80.229118,76.364731,...,2.503401,4.203002,45.720001,41.940001,43.609999,53.209999,55.139999,54.290001,21.358958,75.625
253,CHN,2019,77.6,61.9,88.4,100.0,100.0,100.0,80.229118,76.364731,...,2.290177,4.203002,44.760000,40.819999,42.540000,54.150002,56.279999,55.340000,21.358958,75.625
254,CHN,2020,79.4,65.2,89.4,100.0,100.0,100.0,80.229118,76.364731,...,2.066047,4.203002,44.760000,40.819999,42.540000,54.150002,56.279999,55.340000,21.358958,75.625


Afterwards, we have scaled the values. The escalation process has been done dividing each value by the initial one of an indicator (value in 1990). Considering the start point as 1 (initial value divided by itself), each result will show the growth respect to the initial data.

In [33]:
columns=data.columns.values.tolist()

In [34]:
datae=data.loc[data.loc[:, 'Country'] == country_list[0]]
for i in range(2,len(columns)):
    a=columns[i]
    datae[a]=datae[a]/datae.iloc[0,i]
datau=datae

In [35]:
for u in range(1,len(country_list)):
    datae=data.loc[data.loc[:, 'Country'] == country_list[u]]   
    for i in range(2,len(columns)):
        a=columns[i]
        datae[a]=datae[a]/datae.iloc[0,i]
    datau=pd.concat((datau, datae), axis = 0)
datau

Indicator,Country,Year,Access to clean fuels and technologies for cooking (% of population),"Access to clean fuels and technologies for cooking, rural (% of rural population)","Access to clean fuels and technologies for cooking, urban (% of urban population)",Access to electricity (% of population),"Access to electricity, rural (% of rural population)","Access to electricity, urban (% of urban population)",Account ownership at a financial institution or with a mobile-money-service provider (% of population ages 15+),"Account ownership at a financial institution or with a mobile-money-service provider, female (% of population ages 15+)",...,Urban population growth (annual %),Urban population living in areas where elevation is below 5 meters (% of total population),"Vulnerable employment, female (% of female employment) (modeled ILO estimate)","Vulnerable employment, male (% of male employment) (modeled ILO estimate)","Vulnerable employment, total (% of total employment) (modeled ILO estimate)","Wage and salaried workers, female (% of female employment) (modeled ILO estimate)","Wage and salaried workers, male (% of male employment) (modeled ILO estimate)","Wage and salaried workers, total (% of total employment) (modeled ILO estimate)","Water productivity, total (constant 2015 US$ GDP per cubic meter of total freshwater withdrawal)",Women Business and the Law Index Score (scale 1-100)
352,DEU,1990,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
353,DEU,1991,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,...,0.885023,0.999209,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
354,DEU,1992,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,...,0.837277,0.998418,1.007018,1.027083,1.019342,0.998479,0.994164,0.996015,1.000000,1.000000
355,DEU,1993,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,...,0.798935,0.997627,1.026316,1.050000,1.040619,0.995872,0.990349,0.992693,1.027882,1.000000
356,DEU,1994,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,...,0.602296,0.996836,0.984210,1.083333,1.038685,0.995437,0.984626,0.989372,1.055765,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,CHN,2017,1.742857,2.348936,1.261156,1.030696,1.048707,1.0,1.257169,1.272562,...,0.635700,1.362819,0.707250,0.617275,0.656204,1.555786,1.984976,1.768798,8.610987,1.273684
252,CHN,2018,1.800000,2.510638,1.278713,1.030696,1.048707,1.0,1.257169,1.272562,...,0.580879,1.362819,0.694938,0.602759,0.642647,1.578932,2.020520,1.798278,8.610987,1.273684
253,CHN,2019,1.847619,2.634043,1.293343,1.030696,1.048707,1.0,1.257169,1.272562,...,0.531403,1.362819,0.680347,0.586663,0.626879,1.606825,2.062294,1.833057,8.610987,1.273684
254,CHN,2020,1.890476,2.774468,1.307974,1.030696,1.048707,1.0,1.257169,1.272562,...,0.479397,1.362819,0.680347,0.586663,0.626879,1.606825,2.062294,1.833057,8.610987,1.273684


As later on we want to study the correlations with time moved, we need to create new columns for it. The reason why is because, maybe the effect of a variable does not happen until a couple of years later. The time movements that have been considered are those of the Fibonacci serie within our time period. 

The following pictures helps to realize the behaviour that we were explaining. 

![](https://raw.githubusercontent.com/devonfw-forge/python-data-driven-decisions/main-the-big-three/Logos/Time%20moved.JPG)

In [36]:
shifted=pd.DataFrame()
for i in range(0,len(country_list)):
    dat=datau.loc[datau.loc[:, 'Country'] == country_list[i]]
    dat['GDP (current US$)+1']=dat['GDP (current US$)'].shift(periods=1)
    dat['GDP (current US$)+2']=dat['GDP (current US$)'].shift(periods=2)
    dat['GDP (current US$)+3']=dat['GDP (current US$)'].shift(periods=3)
    dat['GDP (current US$)+5']=dat['GDP (current US$)'].shift(periods=5)
    dat['GDP (current US$)+8']=dat['GDP (current US$)'].shift(periods=8)
    dat['GDP (current US$)+13']=dat['GDP (current US$)'].shift(periods=13)
    dat['GDP (current US$)+21']=dat['GDP (current US$)'].shift(periods=21)
    shifted=pd.concat((shifted, dat), axis = 0)
shifted

Indicator,Country,Year,Access to clean fuels and technologies for cooking (% of population),"Access to clean fuels and technologies for cooking, rural (% of rural population)","Access to clean fuels and technologies for cooking, urban (% of urban population)",Access to electricity (% of population),"Access to electricity, rural (% of rural population)","Access to electricity, urban (% of urban population)",Account ownership at a financial institution or with a mobile-money-service provider (% of population ages 15+),"Account ownership at a financial institution or with a mobile-money-service provider, female (% of population ages 15+)",...,"Wage and salaried workers, total (% of total employment) (modeled ILO estimate)","Water productivity, total (constant 2015 US$ GDP per cubic meter of total freshwater withdrawal)",Women Business and the Law Index Score (scale 1-100),GDP (current US$)+1,GDP (current US$)+2,GDP (current US$)+3,GDP (current US$)+5,GDP (current US$)+8,GDP (current US$)+13,GDP (current US$)+21
352,DEU,1990,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,...,1.000000,1.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
353,DEU,1991,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN
354,DEU,1992,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,...,0.996015,1.000000,1.000000,1.054905,1.000000,NaN,NaN,NaN,NaN,NaN
355,DEU,1993,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,...,0.992693,1.027882,1.000000,1.203142,1.054905,1.000000,NaN,NaN,NaN,NaN
356,DEU,1994,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,...,0.989372,1.055765,1.000000,1.169136,1.203142,1.054905,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,CHN,2017,1.742857,2.348936,1.261156,1.030696,1.048707,1.0,1.257169,1.272562,...,1.768798,8.610987,1.273684,31.129362,30.653486,29.029938,23.644292,14.137706,5.418606,2.393592
252,CHN,2018,1.800000,2.510638,1.278713,1.030696,1.048707,1.0,1.257169,1.272562,...,1.798278,8.610987,1.273684,34.114284,31.129362,30.653486,26.521259,16.868589,6.334809,2.664772
253,CHN,2019,1.847619,2.634043,1.293343,1.030696,1.048707,1.0,1.257169,1.272562,...,1.833057,8.610987,1.273684,38.504955,34.114284,31.129362,29.029938,20.926519,7.626636,2.851657
254,CHN,2020,1.890476,2.774468,1.307974,1.030696,1.048707,1.0,1.257169,1.272562,...,1.833057,8.610987,1.273684,39.572189,38.504955,34.114284,30.653486,23.644292,9.838617,3.031657


In [37]:
data=shifted

For the next part of analyzing this data, we think it is gonna be interesting to have it classify by the categories of the Country groups defined before, to which we call "Continent". This category is useful as it groups the nations with similar economies or geographical proximity, so we can extract common conclusions from them.

We create a dictionary with the regions and the countries included in each one. Where we will relate the countries and regions so then we can apply the .map function and arrive to the final dataframe.

In [38]:
countries_by_region = {
    "Europe": ('DEU','FRA','SWE','GBR','ESP','HRV','POL','GRC','AUT','NLD'),
    'Persian Gulf': ('IRQ','QAT','ARE','SAU','AZE','YEM','YDR','OMN'),
    'North Africa':('DZA','EGY','LBY','ISR','TUR','MAR'),
    'South Africa':('SEN','ZAF','LBR','MOZ','CMR','NGA','GHA'),
    'Asia':('BGD','IND','VNM','THA','IDN','PHL','KOR'),
    'Latam':('MEX','BRA','ARG','PER','VEN','COL','CHL','PAN','CRI'),
    'Pair':('USA','CHN')
    }

all_countries = {}
for region in countries_by_region.keys():
  for country in countries_by_region[region]:
    all_countries[country] = region

print(all_countries)

{'DEU': 'Europe', 'FRA': 'Europe', 'SWE': 'Europe', 'GBR': 'Europe', 'ESP': 'Europe', 'HRV': 'Europe', 'POL': 'Europe', 'GRC': 'Europe', 'AUT': 'Europe', 'NLD': 'Europe', 'IRQ': 'Persian Gulf', 'QAT': 'Persian Gulf', 'ARE': 'Persian Gulf', 'SAU': 'Persian Gulf', 'AZE': 'Persian Gulf', 'YEM': 'Persian Gulf', 'YDR': 'Persian Gulf', 'OMN': 'Persian Gulf', 'DZA': 'North Africa', 'EGY': 'North Africa', 'LBY': 'North Africa', 'ISR': 'North Africa', 'TUR': 'North Africa', 'MAR': 'North Africa', 'SEN': 'South Africa', 'ZAF': 'South Africa', 'LBR': 'South Africa', 'MOZ': 'South Africa', 'CMR': 'South Africa', 'NGA': 'South Africa', 'GHA': 'South Africa', 'BGD': 'Asia', 'IND': 'Asia', 'VNM': 'Asia', 'THA': 'Asia', 'IDN': 'Asia', 'PHL': 'Asia', 'KOR': 'Asia', 'MEX': 'Latam', 'BRA': 'Latam', 'ARG': 'Latam', 'PER': 'Latam', 'VEN': 'Latam', 'COL': 'Latam', 'CHL': 'Latam', 'PAN': 'Latam', 'CRI': 'Latam', 'USA': 'Pair', 'CHN': 'Pair'}


In [39]:
data['Continent']=data['Country'].map(all_countries)
Goldendataframe=data
Goldendataframe

Indicator,Country,Year,Access to clean fuels and technologies for cooking (% of population),"Access to clean fuels and technologies for cooking, rural (% of rural population)","Access to clean fuels and technologies for cooking, urban (% of urban population)",Access to electricity (% of population),"Access to electricity, rural (% of rural population)","Access to electricity, urban (% of urban population)",Account ownership at a financial institution or with a mobile-money-service provider (% of population ages 15+),"Account ownership at a financial institution or with a mobile-money-service provider, female (% of population ages 15+)",...,"Water productivity, total (constant 2015 US$ GDP per cubic meter of total freshwater withdrawal)",Women Business and the Law Index Score (scale 1-100),GDP (current US$)+1,GDP (current US$)+2,GDP (current US$)+3,GDP (current US$)+5,GDP (current US$)+8,GDP (current US$)+13,GDP (current US$)+21,Continent
352,DEU,1990,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,...,1.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Europe
353,DEU,1991,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,...,1.000000,1.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,Europe
354,DEU,1992,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,...,1.000000,1.000000,1.054905,1.000000,NaN,NaN,NaN,NaN,NaN,Europe
355,DEU,1993,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,...,1.027882,1.000000,1.203142,1.054905,1.000000,NaN,NaN,NaN,NaN,Europe
356,DEU,1994,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,...,1.055765,1.000000,1.169136,1.203142,1.054905,NaN,NaN,NaN,NaN,Europe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,CHN,2017,1.742857,2.348936,1.261156,1.030696,1.048707,1.0,1.257169,1.272562,...,8.610987,1.273684,31.129362,30.653486,29.029938,23.644292,14.137706,5.418606,2.393592,Pair
252,CHN,2018,1.800000,2.510638,1.278713,1.030696,1.048707,1.0,1.257169,1.272562,...,8.610987,1.273684,34.114284,31.129362,30.653486,26.521259,16.868589,6.334809,2.664772,Pair
253,CHN,2019,1.847619,2.634043,1.293343,1.030696,1.048707,1.0,1.257169,1.272562,...,8.610987,1.273684,38.504955,34.114284,31.129362,29.029938,20.926519,7.626636,2.851657,Pair
254,CHN,2020,1.890476,2.774468,1.307974,1.030696,1.048707,1.0,1.257169,1.272562,...,8.610987,1.273684,39.572189,38.504955,34.114284,30.653486,23.644292,9.838617,3.031657,Pair


With that all, we export our dataframe all-in-one and by the continent category.

In [40]:
Goldendataframe.to_csv(os.getcwd()+'/Data/GoldenDataFrame.csv')

In [41]:
for region, data in Goldendataframe.groupby('Continent'):
   data.to_csv(os.getcwd()+'/Data/{}.csv'.format(region))

# Categorization of variables

In this section, we are going to attempt a categorization of the whole of the variables, which most of them come the same sources and just differ in the units that are measured, or the total that they are refering, between others. For a simpler treatment of the data, the variables have been pivoted into the same column.

In [42]:
columns_golden=list(Goldendataframe.columns)
del columns_golden[0:2]

In [43]:
Categorization=Goldendataframe.set_index(['Country','Year', 'Continent']).stack().reset_index()
Categorization['Short indicator']=Categorization['Indicator']
Categorization

,Country,Year,Continent,Indicator,0,Short indicator
0,DEU,1990,Europe,Access to clean fuels and technologies for coo...,1.000000,Access to clean fuels and technologies for coo...
1,DEU,1990,Europe,Access to clean fuels and technologies for coo...,1.000000,Access to clean fuels and technologies for coo...
2,DEU,1990,Europe,Access to clean fuels and technologies for coo...,1.000000,Access to clean fuels and technologies for coo...
3,DEU,1990,Europe,Access to electricity (% of population),1.000000,Access to electricity (% of population)
4,DEU,1990,Europe,"Access to electricity, rural (% of rural popul...",1.000000,"Access to electricity, rural (% of rural popul..."
...,...,...,...,...,...,...
1540358,CHN,2021,Pair,GDP (current US$)+3,38.504955,GDP (current US$)+3
1540359,CHN,2021,Pair,GDP (current US$)+5,31.129362,GDP (current US$)+5
1540360,CHN,2021,Pair,GDP (current US$)+8,26.521259,GDP (current US$)+8
1540361,CHN,2021,Pair,GDP (current US$)+13,12.731623,GDP (current US$)+13


 There are some indicators which represent exactly the same through different units, so, we are going to select only one type. For example, in monetary cases, indicators which are expressed with current US $ has been selected. Then, which are showed with the percentage and the total value, we have programmed to selct which ones which show a greater value.

The links used to learn about these functions have been:

https://www.geeksforgeeks.org/how-to-drop-rows-that-contain-a-specific-string-in-pandas/ 

https://www.statology.org/pandas-drop-rows-that-contain-string/ 

In [44]:
import re
discard=["annual % growth","constant 2015 US[$]","% of GNI","constant LCU","current LCU"]
Categorization2=Categorization[~Categorization['Short indicator'].str.contains('|'.join(discard))]
Categorization2

,Country,Year,Continent,Indicator,0,Short indicator
0,DEU,1990,Europe,Access to clean fuels and technologies for coo...,1.000000,Access to clean fuels and technologies for coo...
1,DEU,1990,Europe,Access to clean fuels and technologies for coo...,1.000000,Access to clean fuels and technologies for coo...
2,DEU,1990,Europe,Access to clean fuels and technologies for coo...,1.000000,Access to clean fuels and technologies for coo...
3,DEU,1990,Europe,Access to electricity (% of population),1.000000,Access to electricity (% of population)
4,DEU,1990,Europe,"Access to electricity, rural (% of rural popul...",1.000000,"Access to electricity, rural (% of rural popul..."
...,...,...,...,...,...,...
1540358,CHN,2021,Pair,GDP (current US$)+3,38.504955,GDP (current US$)+3
1540359,CHN,2021,Pair,GDP (current US$)+5,31.129362,GDP (current US$)+5
1540360,CHN,2021,Pair,GDP (current US$)+8,26.521259,GDP (current US$)+8
1540361,CHN,2021,Pair,GDP (current US$)+13,12.731623,GDP (current US$)+13


To check previous step.

In [45]:
#Categorization2.apply(lambda row: row.astype(str).str.contains('US').any(), axis=1)

Now we are going to structure the indicators in a same way to work better. The first step consist of making a new column that shows the units of each variable. Units are showed inside the parenthesis of the indicator name.

In [46]:
Categorization2['Units']=Categorization2['Short indicator'].str.extract(' (\(.*\))')
Categorization2

,Country,Year,Continent,Indicator,0,Short indicator,Units
0,DEU,1990,Europe,Access to clean fuels and technologies for coo...,1.000000,Access to clean fuels and technologies for coo...,(% of population)
1,DEU,1990,Europe,Access to clean fuels and technologies for coo...,1.000000,Access to clean fuels and technologies for coo...,(% of rural population)
2,DEU,1990,Europe,Access to clean fuels and technologies for coo...,1.000000,Access to clean fuels and technologies for coo...,(% of urban population)
3,DEU,1990,Europe,Access to electricity (% of population),1.000000,Access to electricity (% of population),(% of population)
4,DEU,1990,Europe,"Access to electricity, rural (% of rural popul...",1.000000,"Access to electricity, rural (% of rural popul...",(% of rural population)
...,...,...,...,...,...,...,...
1540358,CHN,2021,Pair,GDP (current US$)+3,38.504955,GDP (current US$)+3,(current US$)
1540359,CHN,2021,Pair,GDP (current US$)+5,31.129362,GDP (current US$)+5,(current US$)
1540360,CHN,2021,Pair,GDP (current US$)+8,26.521259,GDP (current US$)+8,(current US$)
1540361,CHN,2021,Pair,GDP (current US$)+13,12.731623,GDP (current US$)+13,(current US$)


Now, short indicator refers to the original indicator name without the units. The extracted information from the origin column has been deleted.

In [47]:
Categorization2['Short indicator']=Categorization2['Short indicator'].str.replace(r" (\(.*\))","")
Categorization2

,Country,Year,Continent,Indicator,0,Short indicator,Units
0,DEU,1990,Europe,Access to clean fuels and technologies for coo...,1.000000,Access to clean fuels and technologies for coo...,(% of population)
1,DEU,1990,Europe,Access to clean fuels and technologies for coo...,1.000000,Access to clean fuels and technologies for coo...,(% of rural population)
2,DEU,1990,Europe,Access to clean fuels and technologies for coo...,1.000000,Access to clean fuels and technologies for coo...,(% of urban population)
3,DEU,1990,Europe,Access to electricity (% of population),1.000000,Access to electricity,(% of population)
4,DEU,1990,Europe,"Access to electricity, rural (% of rural popul...",1.000000,"Access to electricity, rural",(% of rural population)
...,...,...,...,...,...,...,...
1540358,CHN,2021,Pair,GDP (current US$)+3,38.504955,GDP+3,(current US$)
1540359,CHN,2021,Pair,GDP (current US$)+5,31.129362,GDP+5,(current US$)
1540360,CHN,2021,Pair,GDP (current US$)+8,26.521259,GDP+8,(current US$)
1540361,CHN,2021,Pair,GDP (current US$)+13,12.731623,GDP+13,(current US$)


In some cases there are extra information in indicators name. The information of the second parenthesis is extracted as a new column too.

In [48]:
two_parent=Categorization2[Categorization2['Short indicator'].str.contains('Contributing family workers')]
two_parent

,Country,Year,Continent,Indicator,0,Short indicator,Units
151,DEU,1990,Europe,"Contributing family workers, female (% of fema...",1.000000,"Contributing family workers, female",(% of female employment) (modeled ILO estimate)
152,DEU,1990,Europe,"Contributing family workers, male (% of male e...",1.000000,"Contributing family workers, male",(% of male employment) (modeled ILO estimate)
153,DEU,1990,Europe,"Contributing family workers, total (% of total...",1.000000,"Contributing family workers, total",(% of total employment) (modeled ILO estimate)
1151,DEU,1991,Europe,"Contributing family workers, female (% of fema...",1.000000,"Contributing family workers, female",(% of female employment) (modeled ILO estimate)
1152,DEU,1991,Europe,"Contributing family workers, male (% of male e...",1.000000,"Contributing family workers, male",(% of male employment) (modeled ILO estimate)
...,...,...,...,...,...,...,...
1538535,CHN,2020,Pair,"Contributing family workers, male (% of male e...",0.252894,"Contributing family workers, male",(% of male employment) (modeled ILO estimate)
1538536,CHN,2020,Pair,"Contributing family workers, total (% of total...",0.329385,"Contributing family workers, total",(% of total employment) (modeled ILO estimate)
1539524,CHN,2021,Pair,"Contributing family workers, female (% of fema...",0.386565,"Contributing family workers, female",(% of female employment) (modeled ILO estimate)
1539525,CHN,2021,Pair,"Contributing family workers, male (% of male e...",0.252894,"Contributing family workers, male",(% of male employment) (modeled ILO estimate)


Moreover, there are some inidcators with an extra parenthesis adding some  more information. As this information isn't related with units, another column named as 'other specification' has been created.

In [49]:
Categorization2[['Units','Other specification']]=Categorization2['Units'].str.split("\) ", n=1,expand=True)
Categorization2

,Country,Year,Continent,Indicator,0,Short indicator,Units,Other specification
0,DEU,1990,Europe,Access to clean fuels and technologies for coo...,1.000000,Access to clean fuels and technologies for coo...,(% of population),None
1,DEU,1990,Europe,Access to clean fuels and technologies for coo...,1.000000,Access to clean fuels and technologies for coo...,(% of rural population),None
2,DEU,1990,Europe,Access to clean fuels and technologies for coo...,1.000000,Access to clean fuels and technologies for coo...,(% of urban population),None
3,DEU,1990,Europe,Access to electricity (% of population),1.000000,Access to electricity,(% of population),None
4,DEU,1990,Europe,"Access to electricity, rural (% of rural popul...",1.000000,"Access to electricity, rural",(% of rural population),None
...,...,...,...,...,...,...,...,...
1540358,CHN,2021,Pair,GDP (current US$)+3,38.504955,GDP+3,(current US$),None
1540359,CHN,2021,Pair,GDP (current US$)+5,31.129362,GDP+5,(current US$),None
1540360,CHN,2021,Pair,GDP (current US$)+8,26.521259,GDP+8,(current US$),None
1540361,CHN,2021,Pair,GDP (current US$)+13,12.731623,GDP+13,(current US$),None


At the end of the variable name, separated by the last "," it is informing us about to which subgroup makes reference the variable. Thus, there are some indicators that have information divided for small groups. This information is shown as a new column named 'Subgroup'.

In [50]:
Categorization2[['Subgroup']]=Categorization2['Short indicator'].str.extract(',(?P<field>[^,]*?)$')
Categorization2

,Country,Year,Continent,Indicator,0,Short indicator,Units,Other specification,Subgroup
0,DEU,1990,Europe,Access to clean fuels and technologies for coo...,1.000000,Access to clean fuels and technologies for coo...,(% of population),None,NaN
1,DEU,1990,Europe,Access to clean fuels and technologies for coo...,1.000000,Access to clean fuels and technologies for coo...,(% of rural population),None,rural
2,DEU,1990,Europe,Access to clean fuels and technologies for coo...,1.000000,Access to clean fuels and technologies for coo...,(% of urban population),None,urban
3,DEU,1990,Europe,Access to electricity (% of population),1.000000,Access to electricity,(% of population),None,NaN
4,DEU,1990,Europe,"Access to electricity, rural (% of rural popul...",1.000000,"Access to electricity, rural",(% of rural population),None,rural
...,...,...,...,...,...,...,...,...,...
1540358,CHN,2021,Pair,GDP (current US$)+3,38.504955,GDP+3,(current US$),None,NaN
1540359,CHN,2021,Pair,GDP (current US$)+5,31.129362,GDP+5,(current US$),None,NaN
1540360,CHN,2021,Pair,GDP (current US$)+8,26.521259,GDP+8,(current US$),None,NaN
1540361,CHN,2021,Pair,GDP (current US$)+13,12.731623,GDP+13,(current US$),None,NaN


As before, information which is shown as a new column is deleted from the origin one.

In [51]:
Categorization2['Short indicator']=Categorization2['Short indicator'].str.replace(',(?P<field>[^,]*?)$',"")
Categorization2

,Country,Year,Continent,Indicator,0,Short indicator,Units,Other specification,Subgroup
0,DEU,1990,Europe,Access to clean fuels and technologies for coo...,1.000000,Access to clean fuels and technologies for coo...,(% of population),None,NaN
1,DEU,1990,Europe,Access to clean fuels and technologies for coo...,1.000000,Access to clean fuels and technologies for coo...,(% of rural population),None,rural
2,DEU,1990,Europe,Access to clean fuels and technologies for coo...,1.000000,Access to clean fuels and technologies for coo...,(% of urban population),None,urban
3,DEU,1990,Europe,Access to electricity (% of population),1.000000,Access to electricity,(% of population),None,NaN
4,DEU,1990,Europe,"Access to electricity, rural (% of rural popul...",1.000000,Access to electricity,(% of rural population),None,rural
...,...,...,...,...,...,...,...,...,...
1540358,CHN,2021,Pair,GDP (current US$)+3,38.504955,GDP+3,(current US$),None,NaN
1540359,CHN,2021,Pair,GDP (current US$)+5,31.129362,GDP+5,(current US$),None,NaN
1540360,CHN,2021,Pair,GDP (current US$)+8,26.521259,GDP+8,(current US$),None,NaN
1540361,CHN,2021,Pair,GDP (current US$)+13,12.731623,GDP+13,(current US$),None,NaN


All the indicators don't have these elements. So, a checking point is needed.

In [52]:
Categorization2['Subgroup']=Categorization2['Subgroup'].replace(['None'],['total'])
Categorization2['Subgroup']=Categorization2['Subgroup'].fillna('total')

There are some duplicate variables which should be removed too.

In [53]:
Categorization2.drop_duplicates(subset=['Country','Year','Short indicator','Continent','Subgroup'], keep='first')

,Country,Year,Continent,Indicator,0,Short indicator,Units,Other specification,Subgroup
0,DEU,1990,Europe,Access to clean fuels and technologies for coo...,1.000000,Access to clean fuels and technologies for coo...,(% of population),None,total
1,DEU,1990,Europe,Access to clean fuels and technologies for coo...,1.000000,Access to clean fuels and technologies for coo...,(% of rural population),None,rural
2,DEU,1990,Europe,Access to clean fuels and technologies for coo...,1.000000,Access to clean fuels and technologies for coo...,(% of urban population),None,urban
3,DEU,1990,Europe,Access to electricity (% of population),1.000000,Access to electricity,(% of population),None,total
4,DEU,1990,Europe,"Access to electricity, rural (% of rural popul...",1.000000,Access to electricity,(% of rural population),None,rural
...,...,...,...,...,...,...,...,...,...
1540358,CHN,2021,Pair,GDP (current US$)+3,38.504955,GDP+3,(current US$),None,total
1540359,CHN,2021,Pair,GDP (current US$)+5,31.129362,GDP+5,(current US$),None,total
1540360,CHN,2021,Pair,GDP (current US$)+8,26.521259,GDP+8,(current US$),None,total
1540361,CHN,2021,Pair,GDP (current US$)+13,12.731623,GDP+13,(current US$),None,total


Reordering columns, categorization3 is our df after all these division in categories.

In [54]:
Categorization2.rename(columns={Categorization2.columns[4]:'Value'},inplace=True)
Categorization3=Categorization2[['Country','Year','Continent','Indicator','Short indicator','Value','Subgroup','Units','Other specification']]
Categorization3

,Country,Year,Continent,Indicator,Short indicator,Value,Subgroup,Units,Other specification
0,DEU,1990,Europe,Access to clean fuels and technologies for coo...,Access to clean fuels and technologies for coo...,1.000000,total,(% of population),None
1,DEU,1990,Europe,Access to clean fuels and technologies for coo...,Access to clean fuels and technologies for coo...,1.000000,rural,(% of rural population),None
2,DEU,1990,Europe,Access to clean fuels and technologies for coo...,Access to clean fuels and technologies for coo...,1.000000,urban,(% of urban population),None
3,DEU,1990,Europe,Access to electricity (% of population),Access to electricity,1.000000,total,(% of population),None
4,DEU,1990,Europe,"Access to electricity, rural (% of rural popul...",Access to electricity,1.000000,rural,(% of rural population),None
...,...,...,...,...,...,...,...,...,...
1540358,CHN,2021,Pair,GDP (current US$)+3,GDP+3,38.504955,total,(current US$),None
1540359,CHN,2021,Pair,GDP (current US$)+5,GDP+5,31.129362,total,(current US$),None
1540360,CHN,2021,Pair,GDP (current US$)+8,GDP+8,26.521259,total,(current US$),None
1540361,CHN,2021,Pair,GDP (current US$)+13,GDP+13,12.731623,total,(current US$),None


------------------------------

In [55]:
Categorization4=Categorization3.loc[Categorization3['Subgroup']=='total']
Categorization4

,Country,Year,Continent,Indicator,Short indicator,Value,Subgroup,Units,Other specification
0,DEU,1990,Europe,Access to clean fuels and technologies for coo...,Access to clean fuels and technologies for coo...,1.000000,total,(% of population),None
3,DEU,1990,Europe,Access to electricity (% of population),Access to electricity,1.000000,total,(% of population),None
6,DEU,1990,Europe,Account ownership at a financial institution o...,Account ownership at a financial institution o...,1.000000,total,(% of population ages 15+),None
20,DEU,1990,Europe,Adjusted net national income (current US$),Adjusted net national income,1.000000,total,(current US$),None
23,DEU,1990,Europe,Adjusted net national income per capita (curre...,Adjusted net national income per capita,1.000000,total,(current US$),None
...,...,...,...,...,...,...,...,...,...
1540358,CHN,2021,Pair,GDP (current US$)+3,GDP+3,38.504955,total,(current US$),None
1540359,CHN,2021,Pair,GDP (current US$)+5,GDP+5,31.129362,total,(current US$),None
1540360,CHN,2021,Pair,GDP (current US$)+8,GDP+8,26.521259,total,(current US$),None
1540361,CHN,2021,Pair,GDP (current US$)+13,GDP+13,12.731623,total,(current US$),None


In [56]:
Categorization4.to_csv(os.getcwd()+'/Data/Categorization.csv')